In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from kenpompy.utils import login
import numpy as np

In [98]:
convert_dict = {'MIN':int,
               'OREB':int,
               'DREB':int,
               'REB':int,
               'AST':int,
               'STL':int,
               'BLK':int,
               'TO':int,
               'PF':int,
               'PTS':int,
               'FGM':int,
               'FGA':int,
               '3PM':int,
               '3PA':int,
               'FTM':int,
               'FTA':int}
col_order = ['Starters',
                 'PTS',
                 'MIN',
                 'FGM',
                 'FGA',
                 '3PM',
                 '3PA',
                 'FTM',
                 'FTA',
                 'OREB',
                 'DREB',
                 'REB',
                 'AST',
                 'STL',
                 'BLK',
                 'TO',
                 'PF',
                 'Position']

def calculate_possessions(fga,orebs,tos,fta):
    value = (fga-orebs) + tos + (0.475*fta)
    return value

def clean_up_boxscore(df):
    df['Position'] = [i[-1] for i in df.Starters]
    df = df.infer_objects()
    df.Starters = [i[:int((len(i)-1)/2)] for i in df.Starters]
    df['FGM'] = [i[0] for i in df.FG.str.split('-')]
    df['FGA'] = [i[1] for i in df.FG.str.split('-')]
    df['3PM'] = [i[0] for i in df['3PT'].str.split('-')]
    df['3PA'] = [i[1] for i in df['3PT'].str.split('-')]
    df['FTM'] = [i[0] for i in df.FT.str.split('-')]
    df['FTA'] = [i[1] for i in df.FT.str.split('-')]
    df = df.astype(convert_dict)
    df = df.append(df.sum(numeric_only = True),ignore_index = True)
    last_row = len(df) - 1
    df = df[col_order]
    df.iloc[last_row,0] = 'Team'
    df.iloc[last_row,17] = ""
    return df.astype(convert_dict)

In [62]:
url = 'https://www.espn.com/mens-college-basketball/boxscore/_/gameId/401364406'
r = requests.get(url)
soup = BeautifulSoup(r.content,'lxml')

AWAY TEAM IS ALWAYS FIRST

In [63]:
away_team = str(soup.find('title')).split('-')[0].replace("<title>","").strip().split(' vs. ')[0]
home_team = str(soup.find('title')).split('-')[0].replace("<title>","").strip().split(' vs. ')[1]
print(f"Home team is {home_team} and away team is {away_team}")

Home team is Iowa and away team is Purdue


In [99]:
dfs = pd.read_html(url)
dfs[1].columns
dfs[1].columns = [i[0] for i in dfs[1].columns.tolist()]
df = dfs[1].copy()
df = df.dropna()
df = clean_up_boxscore(df)
df

,Starters,PTS,MIN,FGM,FGA,3PM,3PA,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TO,PF,Position
0,M. Gillis,0,11,0,1,0,0,0,2,1,2,3,1,0,0,1,2,F
1,Z. Edey,6,15,2,4,0,0,2,2,1,3,4,1,1,0,1,3,C
2,I. Thompson,18,29,6,8,4,6,2,2,0,1,1,2,0,1,2,2,G
3,E. Hunter Jr.,7,34,2,3,1,2,2,4,0,0,0,2,1,0,0,1,G
4,S. Stefanovic,14,35,5,10,4,9,0,0,0,5,5,4,1,0,3,2,G
5,C. Furst,4,14,2,2,0,0,0,1,1,3,4,2,0,0,0,2,F
6,T. Williams,12,24,6,11,0,0,0,2,3,7,10,5,2,1,3,3,F
7,J. Ivey,15,22,5,7,3,4,2,7,0,5,5,4,1,0,2,2,G
8,E. Morton,7,16,2,3,1,1,2,2,0,0,0,0,1,0,1,0,G
9,Team,83,200,30,49,13,22,10,22,6,26,32,21,7,2,13,17,


In [112]:
test = dfs[2]
type(test.columns[0])

tuple

In [109]:
df

,Starters,PTS,MIN,FGM,FGA,3PM,3PA,FTM,FTA,OREB,DREB,REB,AST,STL,BLK,TO,PF,Position
0,M. Gillis,0,11,0,1,0,0,0,2,1,2,3,1,0,0,1,2,F
1,Z. Edey,6,15,2,4,0,0,2,2,1,3,4,1,1,0,1,3,C
2,I. Thompson,18,29,6,8,4,6,2,2,0,1,1,2,0,1,2,2,G
3,E. Hunter Jr.,7,34,2,3,1,2,2,4,0,0,0,2,1,0,0,1,G
4,S. Stefanovic,14,35,5,10,4,9,0,0,0,5,5,4,1,0,3,2,G
5,C. Furst,4,14,2,2,0,0,0,1,1,3,4,2,0,0,0,2,F
6,T. Williams,12,24,6,11,0,0,0,2,3,7,10,5,2,1,3,3,F
7,J. Ivey,15,22,5,7,3,4,2,7,0,5,5,4,1,0,2,2,G
8,E. Morton,7,16,2,3,1,1,2,2,0,0,0,0,1,0,1,0,G
9,Team,83,200,30,49,13,22,10,22,6,26,32,21,7,2,13,17,
